In [2]:
# Basic Import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Modelling
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge,Lasso
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBRegressor
import warnings

In [51]:
df=pd.read_csv('/content/stud.csv')
df.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [52]:
X = df.drop(columns='writing_score')
X.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score
0,female,group B,bachelor's degree,standard,none,72,72
1,female,group C,some college,standard,completed,69,90
2,female,group B,master's degree,standard,none,90,95
3,male,group A,associate's degree,free/reduced,none,47,57
4,male,group C,some college,standard,none,76,78


In [53]:
y=df['writing_score']
y

,writing_score
0,74
1,88
2,93
3,44
4,75
...,...
995,95
996,55
997,65
998,77


In [54]:
num_cols=X.select_dtypes(exclude='object').columns
cat_cols=X.select_dtypes(include='object').columns

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

num_trf = StandardScaler()
cat_trf = OneHotEncoder(handle_unknown='ignore')

pre_pro = ColumnTransformer([
    ('numerical',num_trf,num_cols),
    ('categorical',cat_trf,cat_cols)
])

In [55]:
X=pre_pro.fit_transform(X)
X.shape

(1000, 19)

In [56]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)
X_train.shape, X_test.shape

((800, 19), (200, 19))

## **Function that returns all the metrics**

In [57]:
def metrics(true,predicted):
  mae=mean_absolute_error(true,predicted)
  mse=mean_squared_error(true,predicted)
  rmse=np.sqrt(mean_squared_error(true,predicted))
  r2=r2_score(true,predicted)
  return mae,mse,rmse,r2

In [58]:
models = {
    "Linear Regression": LinearRegression(),
    "Lasso": Lasso(),
    "Ridge": Ridge(),
    "K-Neighbors Regressor": KNeighborsRegressor(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest Regressor": RandomForestRegressor(),
    "XGBRegressor": XGBRegressor(),
    "AdaBoost Regressor": AdaBoostRegressor()
}
model_list = []
r2_list =[]

for name,model in models.items():
  model.fit(X_train,y_train)
  y_train_pred = model.predict(X_train)
  y_test_pred = model.predict(X_test)
  mae_train,mse_train,rmse_train,r2_train=metrics(y_train,y_train_pred)
  mae_test,mse_test,rmse_test,r2_test=metrics(y_test,y_test_pred)

  print(name)
  model_list.append(name)
  print("Model performance for training set:")
  print(f"Mean absolute error: {mae_train}")
  print(f"Mean squared error: {mse_train}")
  print(f"Root mean squared error: {rmse_train}")
  print(f"R2 score: {r2_train}")

  print("-"*50)

  print("Model performance for testing set:")
  print(f"Mean absolute error: {mae_test}")
  print(f"Mean squared error: {mse_test}")
  print(f"Root mean squared error: {rmse_test}")
  print(f"R2 score: {r2_test}")
  r2_list.append(r2_test)
  print('='*50)
  print()

Linear Regression
Model performance for training set:
Mean absolute error: 2.709898492206654
Mean squared error: 11.383483940770752
Root mean squared error: 3.3739418994361405
R2 score: 0.9499149534431213
--------------------------------------------------
Model performance for testing set:
Mean absolute error: 3.200272452120669
Mean squared error: 14.911021437246678
Root mean squared error: 3.861479177368004
R2 score: 0.9381328994057506

Lasso
Model performance for training set:
Mean absolute error: 3.647822200234647
Mean squared error: 20.37153358008171
Root mean squared error: 4.513483530498556
R2 score: 0.9103693374451822
--------------------------------------------------
Model performance for testing set:
Mean absolute error: 3.8963838154751853
Mean squared error: 24.26152957941904
Root mean squared error: 4.925599413210441
R2 score: 0.8993368430608714

Ridge
Model performance for training set:
Mean absolute error: 2.7102066100449553
Mean squared error: 11.383799131447883
Root mean

In [59]:
results=pd.DataFrame(list(zip(model_list,r2_list)),columns=['Model Name','R2_Score']).sort_values(by=['R2_Score'],ascending=False)
results

,Model Name,R2_Score
2,Ridge,0.938143
0,Linear Regression,0.938133
5,Random Forest Regressor,0.917113
6,XGBRegressor,0.916216
7,AdaBoost Regressor,0.911775
1,Lasso,0.899337
3,K-Neighbors Regressor,0.880447
4,Decision Tree,0.862043
